In [15]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

# 解决坐标轴刻度负号乱码
plt.rcParams['axes.unicode_minus'] = False

# 解决中文乱码问题
plt.rcParams['font.sans-serif'] = ['Simhei']
plt.style.use('ggplot')

# 导入数据集
iris = pd.read_csv('./data/iris.txt', header = None)
iris.head()
iris.shape

,0,1,2,3,4
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


(150, 5)

In [16]:
# 1.实现欧氏距离的计算
def distEClud(sampleA, sampleB):
  
  '''
  计算两个向量之间的欧氏距离
  '''
  
  return np.sum(np.power(sampleA - sampleB, 2), axis = 1)

In [17]:
# 2.实现随机生成质心
def randCenter(dataSet, k):
  
  '''
    随机生成质心
    :param dataSet:传入数据集
    :param k:质心的个数
  '''
  
  # 获取数据集中特征的个数
  n = dataSet.shape[1]
  
  # 获取每一个特征值的最小值和最大值
  data_min = dataSet.iloc[:, : n - 1].min()
  data_max = dataSet.iloc[:, : n - 1].max()
  
  # 在最大最小值构成的均匀分布的样本中进行随机抽样，生成指定的k个质心
  return np.random.uniform(data_min, data_max, (k, n - 1))
  

In [19]:
# 实现K-Means聚类函数
def K_means(dataSet, k, distMeans = distEClud, createCent = randCenter):
  
  '''
    K-Means聚类函数
    :param dataSet:传入数据集
    :param k:质心的个数
    :param distMeans:距离计算函数
  '''
  
  # 获取数据集的行数和列数
  m, n = dataSet.shape # m = 150, n = 5
  
  # 生成随机质心 - 第一个中间变量容器对象 - 职责是用于cun'chu最新更新的质心
  centrodis = createCent(dataSet, k) # centrodis变量保存的是三个长度为4的一位数组 - 三位数字结构 - 三行四列
  
  # 此时的centrodis质心是随机生成的
  
  # 接下来构造第二个中间变量容器对象result_set
  # result_set最终构造完毕的结构形态：[数据集(5列), 该样本到最近质心的距离, 本次迭代中该样本最近质心的编号, 上次迭代时该样本最近质心的变化]
  # 最终比对result_set下第七第八列的值是否相等，如果不相等，表示此时所得的质心还不是最优质心；如果相等，表示此时所得的质心即为最优质心
  clusterss = np.zeros((m, 3))
  
  # 初始化第六列的值 - 因为还没有开始聚类算法，所以第六列样本到最近质心的距离先给定无穷大作为初始值
  clusterss[:, 0] = np.inf
  
  # 初始化第七列的值 - 因为还没有开始聚类算法，所以第七、八列样本最近质心的编号先给定-1作为初始值
  clusterss[:, 1 : 3] = -1 
  
  # 将dataSet和centrodis拼接，最终得到result_set
  reslut_set = pd.concat(
    [dataSet, pd.DataFrame(clusterss)],
    axis = 1,
    ignore_index = True
  )
  
  # 开始执行核心算法，即迭代聚类的过程
  # 准备一个参数变量，负责调度整个迭代过程是否需要继续执行，无需参与到实际算法的运算，将其设置一个初值为True，表示此时对应的质心还在变化，聚类迭代过程还需继续执行
  # 如果该参数变量某一时刻被算法逻辑置为False，则表示质心已经稳定，质心不在发生变化，此时算法就应该终止迭代
  clusterChanged = True
  
  while clusterChanged:
    
    # 循环开始执行后，因为本次迭代完成后，不知道是否需要开启下一次迭代(因为是否需要开启下一次迭代的前提条件是本次迭代时的求得的质心与上一次迭代时求得的质心发生了变化)，而现在还没开始一次迭代的过程，不知道质心是否发生了变化，保险起见，循环进来后先把clusterChanged变量的值置为False，如果本次迭代完毕，发现质心发生变化，需要再次开启新一轮，则再将clusterChanged变量的值重新置为True
    clusterChanged = False
  
    # 开始本次聚类迭代过程
    for i in range(m): # 遍历result_set中的每一行样本
      
      # 先算距离
      dist = distMeans(dataSet.iloc[i, : n - 1].values, centrodis) 
      
      # 记录当前第i行样本与k个质心中谁的距离最短
      reslut_set.iloc[i, n] = dist.min()
    
      # 记录求得的距离最近的质心的编号(索引)
      reslut_set.iloc[i, n + 1] = np.where(dist == dist.min())[0] # 最近质心的编号 - 获取后将其替换到result_set中第七列的值
  
      # 本次迭代求得的第i样本最近的质心已经找到，接下来考虑循环要不要开启下一次迭代
      # 判断依据就是：本次迭代求得的最近质心与上一次求得的最近质心作比较，如果相等，就不需要开启，因为相等就表示质心不再发生变化，反之，如果不相等，则还需要继续迭代
      clusterChanged = not (reslut_set.iloc[:, -1] == reslut_set.iloc[:, -2]).all()
      
    # 如果150条样本的质心全部计算完毕，且样本的质心发生变化，此时代表质心还未收敛，分组求均值，更新质心数组，将本次求得的质心编号更新到上一次求得的质心编号 
    if clusterChanged:
      
      # 按照索引为n + 1(质心索引)(第七列)进行分组求均值
      cent_df = reslut_set.groupby(n + 1).mean()
      
      # 重置质心数组centrodis,用最新的质心替换上一次的质心
      centrodis = cent_df.iloc[:, : n - 1].values
  
      # 再将result_set的倒数第一列的值(上一次迭代求得的质心编号)替换为倒数第二列的值(本次迭代求得的质心编号)
      reslut_set.iloc[:, -1] = reslut_set.iloc[:, -2]
      
  return centrodis,reslut_set

if __name__ == '__main__':
  K_means(iris, 3)

  

TypeError: Could not convert Iris-versicolorIris-versicolorIris-versicolorIris-versicolorIris-versicolorIris-versicolorIris-versicolorIris-versicolorIris-versicolorIris-versicolorIris-versicolorIris-versicolorIris-versicolorIris-versicolorIris-versicolorIris-versicolorIris-versicolorIris-versicolorIris-versicolorIris-versicolorIris-versicolorIris-versicolorIris-versicolorIris-versicolorIris-versicolorIris-versicolorIris-versicolorIris-versicolorIris-versicolorIris-versicolorIris-versicolorIris-versicolorIris-versicolorIris-versicolorIris-versicolorIris-versicolorIris-versicolorIris-versicolorIris-versicolorIris-versicolorIris-versicolorIris-versicolorIris-versicolorIris-versicolorIris-versicolorIris-versicolorIris-versicolorIris-virginicaIris-virginicaIris-virginicaIris-virginicaIris-virginicaIris-virginicaIris-virginicaIris-virginicaIris-virginicaIris-virginicaIris-virginicaIris-virginicaIris-virginicaIris-virginicaIris-virginicaIris-virginicaIris-virginicaIris-virginicaIris-virginica